In [99]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

In [100]:
fp = Path('Resources/clean_nfl_data.csv')

df_2 = pd.read_csv(fp)

df_2 = pd.DataFrame(df_2)

df_2

,Unnamed: 0,play_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,...,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,2,55,TEN,ARI,REG,1,TEN,home,ARI,TEN,...,22.0,00-0032764,-1.399805,NaN,NaN,NaN,NaN,NaN,0.491433,-49.143300
1,3,76,TEN,ARI,REG,1,TEN,home,ARI,TEN,...,17.0,00-0029701,0.032412,1.165133,5.803177,4.0,0.896654,0.125098,0.697346,30.265415
2,4,100,TEN,ARI,REG,1,TEN,home,ARI,TEN,...,17.0,00-0029701,-1.532898,0.256036,4.147637,2.0,0.965009,0.965009,0.978253,2.174652
3,6,152,TEN,ARI,REG,1,ARI,away,TEN,ARI,...,1.0,00-0035228,2.692890,0.567838,7.420427,4.0,1.000000,1.000000,0.458989,54.101130
4,7,181,TEN,ARI,REG,1,ARI,away,TEN,TEN,...,1.0,00-0035228,-1.009908,NaN,NaN,NaN,NaN,NaN,0.419833,-41.983326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,50704,3855,CIN,LA,POST,22,CIN,home,LA,CIN,...,9.0,00-0036442,0.644359,0.415150,5.589664,4.0,0.388812,0.237658,0.957691,4.230869
35762,50705,3884,CIN,LA,POST,22,CIN,home,LA,CIN,...,9.0,00-0036442,0.482142,0.420278,4.414923,2.0,0.997858,0.462855,0.961070,3.893024
35763,50706,3908,CIN,LA,POST,22,CIN,home,LA,LA,...,9.0,00-0036442,-0.453694,0.780765,7.801189,4.0,1.000000,1.000000,0.942241,5.775929
35764,50707,3930,CIN,LA,POST,22,CIN,home,LA,LA,...,34.0,00-0033526,-1.242206,NaN,NaN,NaN,NaN,NaN,0.681295,-68.129550


In [101]:
df_2['play_type'] = df_2['play_type'].astype(str).str.replace('run' , '1')
df_2['play_type'] = df_2['play_type'].astype(str).str.replace('pass' , '0')
df_2['play_type'] = df_2['play_type'].astype('float')
df_2

,Unnamed: 0,play_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,...,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,2,55,TEN,ARI,REG,1,TEN,home,ARI,TEN,...,22.0,00-0032764,-1.399805,NaN,NaN,NaN,NaN,NaN,0.491433,-49.143300
1,3,76,TEN,ARI,REG,1,TEN,home,ARI,TEN,...,17.0,00-0029701,0.032412,1.165133,5.803177,4.0,0.896654,0.125098,0.697346,30.265415
2,4,100,TEN,ARI,REG,1,TEN,home,ARI,TEN,...,17.0,00-0029701,-1.532898,0.256036,4.147637,2.0,0.965009,0.965009,0.978253,2.174652
3,6,152,TEN,ARI,REG,1,ARI,away,TEN,ARI,...,1.0,00-0035228,2.692890,0.567838,7.420427,4.0,1.000000,1.000000,0.458989,54.101130
4,7,181,TEN,ARI,REG,1,ARI,away,TEN,TEN,...,1.0,00-0035228,-1.009908,NaN,NaN,NaN,NaN,NaN,0.419833,-41.983326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,50704,3855,CIN,LA,POST,22,CIN,home,LA,CIN,...,9.0,00-0036442,0.644359,0.415150,5.589664,4.0,0.388812,0.237658,0.957691,4.230869
35762,50705,3884,CIN,LA,POST,22,CIN,home,LA,CIN,...,9.0,00-0036442,0.482142,0.420278,4.414923,2.0,0.997858,0.462855,0.961070,3.893024
35763,50706,3908,CIN,LA,POST,22,CIN,home,LA,LA,...,9.0,00-0036442,-0.453694,0.780765,7.801189,4.0,1.000000,1.000000,0.942241,5.775929
35764,50707,3930,CIN,LA,POST,22,CIN,home,LA,LA,...,34.0,00-0033526,-1.242206,NaN,NaN,NaN,NaN,NaN,0.681295,-68.129550


In [102]:

print(df_2.columns.tolist())


['Unnamed: 0', 'play_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 'game_half', 'drive', 'qtr', 'down', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'td_team', 'td_player_name', 'td_player_id', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential_post', 'no_score_prob', 'opp_fg_prob', 'opp_safety_prob', 'opp_td_prob', 'fg_prob', 'safety_prob', 'td_prob', 'ep', 'epa', 'total_home_epa', 'total_away_epa', 'total_home_rush_epa', 'total_away_rush_epa', 'total_home_pass_epa', 'total_away_pass_epa', 'air_epa', 'yac_epa', 'comp_air_epa', 'comp_yac_epa', 'total_home_comp_air_epa', 'total_away_comp_air_epa', 't

In [103]:
df_2 = df_2[['home_team' , 'away_team' , 'posteam' , 'down' , 'ydstogo' , 'play_type']].convert_dtypes()


In [104]:
df_2 = df_2[df_2.down != 0 ]

In [105]:
df_2['down'].isnull().sum()

0

In [106]:

df_2 = df_2[['down' , 'play_type' , 'ydstogo']].convert_dtypes()
df_2

,down,play_type,ydstogo
0,1,1,10
1,2,0,13
2,3,0,10
3,1,0,10
4,1,1,10
...,...,...,...
35761,1,0,10
35762,1,0,10
35763,2,0,1
35764,3,1,1


In [107]:
df_2 = df_2.dropna(axis = 'columns' , how='any')

df_2 = df_2.dropna()

df_2


,down,play_type,ydstogo
0,1,1,10
1,2,0,13
2,3,0,10
3,1,0,10
4,1,1,10
...,...,...,...
35761,1,0,10
35762,1,0,10
35763,2,0,1
35764,3,1,1


In [108]:
x_2 = {'down_1': '1' , 'down_2': '2' , 'down_3': '3' , 'down_4': '4'}
df_2 = df_2.replace(x_2)


In [109]:

df_2.reset_index(inplace=True, drop=True)

df_2.head(10)


,down,play_type,ydstogo
0,1,1,10
1,2,0,13
2,3,0,10
3,1,0,10
4,1,1,10
5,1,0,18
6,2,0,17
7,1,1,9
8,2,0,24
9,3,0,16


In [110]:

X_2 = df_2.drop('play_type' , axis=1)

X_2 = pd.get_dummies(X_2)

# Create our target
y_2 = df_2['play_type'].astype('int')

In [111]:
X_2.describe()

,down,ydstogo
count,35608.000000,35608.000000
mean,1.809958,8.439845
std,0.837031,3.973347
min,1.000000,1.000000
25%,1.000000,6.000000
50%,2.000000,10.000000
75%,2.000000,10.000000
max,4.000000,43.000000


In [112]:
y_2.value_counts()


0    20898
1    14710
Name: play_type, dtype: int64

In [113]:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, random_state=1)
X_2_train.shape

(26706, 2)

In [114]:
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc.fit(X_2_train, y_2_train)
y_2_pred = brfc.predict(X_2_test)

In [115]:
y_2_pred = brfc.predict(X_2_test)

balanced_accuracy_score(y_2_test , y_2_pred)

0.6455510366029948

In [116]:
balanced_accuracy_score(y_2_test , y_2_pred)
confusion_matrix(y_2_test, y_2_pred)
print(classification_report_imbalanced(y_2_test, y_2_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.76      0.53      0.76      0.63      0.64      0.40      5233
          1       0.53      0.76      0.53      0.63      0.64      0.41      3669

avg / total       0.67      0.63      0.67      0.63      0.64      0.40      8902



In [117]:
results_2 = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)
results_2.head(20)

,Prediction,Actual
0,0,0
1,1,0
2,1,0
3,0,0
4,1,1
5,1,1
6,0,0
7,1,0
8,0,0
9,0,1


In [118]:
eec = EasyEnsembleClassifier(n_estimators=100 , random_state=1)
eec = eec.fit(X_2_train , y_2_train)
y_2_pred = eec.predict(X_2_test)


In [119]:
balanced_accuracy_score(y_2_test , y_2_pred)


0.6455792138668388

In [120]:
confusion_matrix(y_2_test , y_2_pred)
print(classification_report_imbalanced(y_2_test , y_2_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.76      0.54      0.76      0.63      0.64      0.40      5233
          1       0.53      0.76      0.54      0.62      0.64      0.41      3669

avg / total       0.66      0.63      0.66      0.63      0.64      0.40      8902



In [121]:
results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,0
1,1,0
2,1,0
3,0,0
4,1,1
5,1,1
6,0,0
7,1,0
8,0,0
9,0,1


## Start on Resampling portion (from MLMockUp) tomorrow 

In [122]:
ros = RandomOverSampler(random_state=1)
X_2_resamp, y_2_resamp = ros.fit_resample(X_2_train, y_2_train)
Counter(y_2_resamp)

Counter({0: 15665, 1: 15665})

In [123]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_2_resamp, y_2_resamp)

LogisticRegression(random_state=1)

In [124]:
y_2_pred = model.predict(X_2_test)
balanced_accuracy_score(y_2_test, y_2_pred)

0.6404460299407126

In [125]:
results = pd.DataFrame({"Prediction": y_2_pred, "Actual": y_2_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,0
1,1,0
2,1,0
3,0,0
4,1,1
5,1,1
6,0,0
7,1,0
8,0,0
9,0,1


In [126]:
matrix = confusion_matrix(y_2_test, y_2_pred)


In [127]:
matrix = confusion_matrix(y_2_test, y_2_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual Run", "Actual Pass"], columns=["Predicted Run", "Predicted Pass"])
cm_df

## Need to figure out a better display output. Confusing as it reads right now. Im not sure which is Predicted and Actual. 

,Predicted Run,Predicted Pass
Actual Run,2832,2401
Actual Pass,955,2714


In [128]:
print(classification_report_imbalanced(y_2_test, y_2_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.54      0.74      0.63      0.63      0.39      5233
          1       0.53      0.74      0.54      0.62      0.63      0.41      3669

avg / total       0.66      0.62      0.66      0.62      0.63      0.40      8902



#### Smote Oversampling 

In [129]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_2_resamp, y_2_resamp)

LogisticRegression(random_state=1)

In [130]:
y_2_pred_s = y_2_pred = model.predict(X_2_test)
acc_score_2 = balanced_accuracy_score(y_2_test, y_2_pred_s)
acc_score_2

0.6404460299407126

In [131]:
matrix_sm = confusion_matrix(y_2_test, y_2_pred_s)

cm2_df = pd.DataFrame(
    matrix_sm, index=["Actual Run", "Actual Pass"], columns=["Predicted Run", "Predicted Pass"])
cm2_df

,Predicted Run,Predicted Pass
Actual Run,2832,2401
Actual Pass,955,2714


#### Undersampling

In [132]:
cc = ClusterCentroids(random_state=1)
X_2_resample3, y_2_resample3 = cc.fit_resample(X_2_train, y_2_train)
Counter(y_2_resample3)


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
  File "_pydevd_bundle/pydevd_cython.pyx", line 1363, in _pydevd_bundle.pydevd_cython.SafeCallWrapper.__call__
  File "_pydevd_bundle/pydevd_cython.pyx", line 1664, in _pydevd_bundle.pydevd_cython.ThreadTracer.__call__
  File "c:\Users\Camer\anaconda3\lib\site-packages\debugpy\_vendored\pydevd\_pydev_bundle\pydev_is_thread_alive.py", line 9, in is_thread_alive
    def is_thread_alive(t):
KeyboardInterrupt: 


In [ ]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_2_resample3, y_2_resample3)
y_2_pred_cc = model.predict(X_2_test)


In [ ]:

acc_score3 = balanced_accuracy_score(y_2_test, y_2_pred_cc)
acc_score3


In [ ]:
matrix_cc = confusion_matrix(y_2_test, y_2_pred_cc)


In [ ]:

cm3_df = pd.DataFrame(
    matrix_cc, index=["Actual Run", "Actual Pass"], columns=["Predicted Run", "Predicted Pass"])
cm3_df
print(classification_report_imbalanced(y_2_test, y_2_pred_cc))
#### Combination (Over and Under) Sampling